### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)])

In [0]:
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)])

In [0]:
person_df = spark.read.json(f"{bronze_folder_path}/{v_file_date}/person.json", schema=persons_schema)

### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "personId" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "envicronment"
3. Agregar la columnas "name" a partir de la concatenacion de forename y surname

In [0]:
from pyspark.sql.functions import current_timestamp, lit, concat, col

In [0]:
person_with_columns_df = person_df\
    .withColumnsRenamed({"personId": "person_id"})\
    .withColumns({"ingestion_date": current_timestamp(), 
                  "environment": lit(v_environment), 
                  "name": concat(col("personName.forename"), lit(" "), col("personName.surname")),
                  "file_date": lit(v_file_date)})

### Paso 3 - Eliminar las columnas no requeridas

In [0]:
persons_final_df = person_with_columns_df.drop(col("personName"))

In [0]:
#overwrite_partition(persons_final_df, "movie_silver", "persons")

In [0]:
merge_condicion = "tgt.person_id = src.person_id AND tgt.file_date = src.file_date"
merge_delta_lake(persons_final_df, "movie_silver", "persons", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.persons
GROUP BY file_date;

In [0]:
%sql  
select * from movie_silver.persons